In [ ]:

import bz2
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import nltk
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import keras


df = pd.read_csv('preprocessed_data 2.csv', header=0)
df_model = df
print(df_model.columns)
df_train, df_test = train_test_split(df_model, test_size=0.2, random_state=42)
print(df_train.columns)

df_train.head(2)
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorize text using TfidfVectorizer
tfidf = TfidfVectorizer(min_df=.01) #(ngram_range=(1,2),min_df=2, max_features = 10000, min_df=.01)
# Convert a collection of raw documents to a matrix of TF-IDF term features. fit_transform() on TRAIN data
train_tfidf_matrix = tfidf.fit_transform(df_train["clean_text_4"].values)

# transform() on TEST data
# Using the transform method we can use the same mean and variance as it is calculated from our training data to transform our test data. Thus, the parameters learned by our model using the training data will help us to transform our test data.
test_tfidf_matrix = tfidf.transform(df_test["clean_text_4"].values)
# Extract feature_names
terms_features = tfidf.get_feature_names_out()
print(len(terms_features))
print(terms_features[0:10])
train_tfidf_vector = train_tfidf_matrix.toarray()
test_tfidf_vector = test_tfidf_matrix.toarray()
result = pd.DataFrame(data=train_tfidf_matrix.toarray(), columns=terms_features)
print(result.head(5))



Index(['rating', 'title', 'text', 'language', 'language_label', 'label',
       'clean_text_1', 'clean_text_2', 'clean_text_4', 'clean_text_5',
       'final_text'],
      dtype='object')
Index(['rating', 'title', 'text', 'language', 'language_label', 'label',
       'clean_text_1', 'clean_text_2', 'clean_text_4', 'clean_text_5',
       'final_text'],
      dtype='object')
672
['able' 'about' 'absolutely' 'acting' 'action' 'actually' 'add' 'after'
 'again' 'against']
   able     about  absolutely  acting  action  actually  add    after  again  \
0   0.0  0.000000         0.0     0.0     0.0       0.0  0.0  0.00000    0.0   
1   0.0  0.114633         0.0     0.0     0.0       0.0  0.0  0.00000    0.0   
2   0.0  0.000000         0.0     0.0     0.0       0.0  0.0  0.00000    0.0   
3   0.0  0.118789         0.0     0.0     0.0       0.0  0.0  0.14214    0.0   
4   0.0  0.000000         0.0     0.0     0.0       0.0  0.0  0.00000    0.0   

   against  ...  written  wrong  year  years  y

In [ ]:
#rnn model

x_train,x_test,y_train,y_test= train_test_split(train_tfidf_vector,df_train['rating'])
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units = 256, activation = 'relu', input_shape=(672,)))
model.add(tf.keras.layers.Dense(units=128, activation='relu',kernel_initializer="he_normal", kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(units=64, activation='relu',kernel_initializer="he_normal", kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4)))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(units=6, activation='softmax'))

In [ ]:
model.compile(optimizer = 'SGD',
             loss= 'sparse_categorical_crossentropy',
             metrics=['sparse_categorical_accuracy'])
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_53 (Dense)            (None, 256)               172288    
                                                                 
 dense_54 (Dense)            (None, 128)               32896     
                                                                 
 dropout_23 (Dropout)        (None, 128)               0         
                                                                 
 dense_55 (Dense)            (None, 64)                8256      
                                                                 
 dropout_24 (Dropout)        (None, 64)                0         
                                                                 
 dense_56 (Dense)            (None, 6)                 390       
                                                                 
Total params: 213830 (835.27 KB)
Trainable params: 21

In [ ]:
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
4676/4676 [==============================] - 21s 4ms/step - loss: 1.6791 - sparse_categorical_accuracy: 0.2432
Epoch 2/10
4676/4676 [==============================] - 22s 5ms/step - loss: 1.4590 - sparse_categorical_accuracy: 0.3853
Epoch 3/10
4676/4676 [==============================] - 20s 4ms/step - loss: 1.3617 - sparse_categorical_accuracy: 0.4384
Epoch 4/10
4676/4676 [==============================] - 20s 4ms/step - loss: 1.3410 - sparse_categorical_accuracy: 0.4474
Epoch 5/10
4676/4676 [==============================] - 23s 5ms/step - loss: 1.3315 - sparse_categorical_accuracy: 0.4516
Epoch 6/10
4676/4676 [==============================] - 20s 4ms/step - loss: 1.3243 - sparse_categorical_accuracy: 0.4541
Epoch 7/10
4676/4676 [==============================] - 21s 5ms/step - loss: 1.3184 - sparse_categorical_accuracy: 0.4582
Epoch 8/10
4676/4676 [==============================] - 20s 4ms/step - loss: 1.3136 - sparse_categorical_accuracy: 0.4586
Epoch 9/10
4676/4676 [==

In [ ]:

y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)


1559/1559 [==============================] - 3s 2ms/step


In [ ]:

print(y_pred)
print(y_test.values)

[1 4 2 ... 5 5 4]
[1 3 2 ... 3 5 3]


In [ ]:

from sklearn.metrics import confusion_matrix, accuracy_score


cn = confusion_matrix(y_test.values,y_pred)
print(cn)

acc_cm = accuracy_score(y_test.values, y_pred)
print(acc_cm)

[[7393 2399 1076  569  918]
 [3226 4359 2991 1110  857]
 [1331 2523 4654 2695 1151]
 [ 780  919 2516 4732 3580]
 [ 897  527  762 2413 7968]]
0.4668463093061303


In [ ]:
#logistic Regression

from sklearn.linear_model import LogisticRegression

model= LogisticRegression(multi_class='multinomial',max_iter=1000, penalty='l2').fit(train_tfidf_vector,df_train['rating'])

# prediction based on model trained

y_pred=model.predict(test_tfidf_vector)

"""
In order to train the model, the method Logistic Regression was implemented. In particular,  multiclass configuration was set as the target variable had more than 3 variables.

Regularization L2, or ridge, was used to prevent overfiting.

This code is performing a logistic regression on the dataset that has been transformed using TF-IDF.
"""
from sklearn.metrics import confusion_matrix, accuracy_score


cn = confusion_matrix(df_test['rating'],y_pred )
print(cn)

acc_cm = accuracy_score(df_test['rating'], y_pred)
print(acc_cm)

0
0
0
[[5898 2009  781  439  722]
 [2617 3522 2243  915  658]
 [1100 2109 3799 2153  906]
 [ 638  766 1960 3694 2856]
 [ 727  399  595 1952 6419]]
0.46779076528259517


In [ ]:
#Multinomial Naive Bayes
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(train_tfidf_vector,df_train['rating'])
preds = nb.predict(test_tfidf_vector)
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(df_test['rating'], preds))
print('\n')
print(classification_report(df_test['rating'], preds))
cn = confusion_matrix(df_test['rating'],preds )
print(cn)

acc_cm = accuracy_score(df_test['rating'], preds)
print(acc_cm)

[[5603 2087  810  559  790]
 [2608 3302 2306 1005  734]
 [1365 1834 3670 2214  984]
 [ 814  791 1979 3499 2831]
 [ 866  470  662 1946 6148]]


              precision    recall  f1-score   support

           1       0.50      0.57      0.53      9849
           2       0.39      0.33      0.36      9955
           3       0.39      0.36      0.38     10067
           4       0.38      0.35      0.37      9914
           5       0.54      0.61      0.57     10092

    accuracy                           0.45     49877
   macro avg       0.44      0.45      0.44     49877
weighted avg       0.44      0.45      0.44     49877

[[5603 2087  810  559  790]
 [2608 3302 2306 1005  734]
 [1365 1834 3670 2214  984]
 [ 814  791 1979 3499 2831]
 [ 866  470  662 1946 6148]]
0.4455360186057702


In [ ]:
#Logistic regression with concatenation

import scipy.sparse as sp
#matrix and dataframe  concatenation
x=sp.vstack((train_tfidf_matrix,test_tfidf_matrix))

y=pd.concat([df_train['rating'], df_test['rating']])

#spliting data
x_train,x_test,y_train,y_test= train_test_split(x,y)

#model training to logistic regression for multiple values and ridge penalty
from sklearn.linear_model import LogisticRegression
model= LogisticRegression(multi_class='multinomial', max_iter=1000, penalty='l2').fit(x_train,y_train)

# prediction based on model trained
print(model.predict_proba(x_test))
y_pred=model.predict(x_test)

"""
In order to train the model, the method Logistic Regression was implemented. In particular,  multiclass configuration was set as the target variable had more than 3 variables.

Regularization L2, or ridge, was used to prevent overfiting.

This code is performing a logistic regression on the dataset that has been transformed using TF-IDF.
"""
from sklearn.metrics import confusion_matrix, accuracy_score


cn = confusion_matrix(y_test,y_pred )
print(cn)

acc_cm = accuracy_score(y_test, y_pred)
print(acc_cm)

[[0.00124018 0.00418158 0.01934015 0.29190451 0.68333358]
 [0.10323997 0.15567941 0.1729058  0.31656558 0.25160925]
 [0.22921159 0.08653493 0.02255524 0.14749463 0.51420362]
 ...
 [0.03332114 0.11380455 0.15135416 0.21212376 0.48939639]
 [0.05560014 0.14368521 0.42281847 0.31805128 0.05984491]
 [0.16890622 0.27957516 0.34870258 0.09585785 0.10695819]]
[[7520 2424 1021  534  909]
 [3270 4363 2837 1144  792]
 [1366 2648 4628 2667 1116]
 [ 830  954 2403 4786 3493]
 [ 869  509  777 2431 8055]]
0.4707920315657781


In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(x_train,y_train)
preds = nb.predict(x_test)



In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import confusion_matrix, classification_report

#Multinomial Naive Bayes with concatenation

print(confusion_matrix(y_test, preds))
print('\n')
print(classification_report(y_test, preds))
cn = confusion_matrix(y_test,preds )
print(cn)
cn = confusion_matrix(y_test,preds )
print(cn)

acc_cm = accuracy_score(y_test, preds)
print(acc_cm)

[[7066 2600 1096  642 1004]
 [3269 4073 2949 1194  921]
 [1652 2327 4512 2677 1257]
 [1051  946 2494 4429 3546]
 [1059  605  828 2409 7740]]


              precision    recall  f1-score   support

           1       0.50      0.57      0.53     12408
           2       0.39      0.33      0.35     12406
           3       0.38      0.36      0.37     12425
           4       0.39      0.36      0.37     12466
           5       0.53      0.61      0.57     12641

    accuracy                           0.45     62346
   macro avg       0.44      0.45      0.44     62346
weighted avg       0.44      0.45      0.44     62346

[[7066 2600 1096  642 1004]
 [3269 4073 2949 1194  921]
 [1652 2327 4512 2677 1257]
 [1051  946 2494 4429 3546]
 [1059  605  828 2409 7740]]
[[7066 2600 1096  642 1004]
 [3269 4073 2949 1194  921]
 [1652 2327 4512 2677 1257]
 [1051  946 2494 4429 3546]
 [1059  605  828 2409 7740]]
0.4462194848105733
